In [12]:
%reload_ext autoreload
%autoreload 2

from func import *
import pandas as pd



In [2]:
filename = 'all_data_anonymized.csv'
date_col = 'todate'
start_date = '1980-01-01'
seperator = ';'
fill = 0
lim = 5
df_read = load_and_preprocess_data(filename, date_col, start_date, seperator, fill, lim)


d:\master3\Master-thesis\Files\func.py:21: DtypeWarning: Columns (15,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df_read = pd.read_csv(filename, sep=seperator)


In [3]:
windows = [5, 10, 20, 40, 60, 100, 180, 240, 360, 480]
window_m = [10, 30, 60, 100, 180]
assets = df_read.columns
df_feat = add_features(df_read, window_m)

In [4]:
target_days = 9

return_column_shift = 'avgreturn'
volatility_column_shift = 'volatility'
df = add_y_col(df_feat, df_read, date_col, target_days, return_column_shift, volatility_column_shift)

In [5]:
X_train, X_eval, X_test, y_train, y_eval, y_test, X_train_eval, y_train_eval = prepare_training_dataset(df, date_col, shuffle=False, train_split=0.25, eval_split=0.25)

In [6]:
param_grid_alpha = {'alpha': [0.1, 1.0, 10.0]}
ridge_best, grid_search = optimize_and_train_ridge(X_train, y_train, X_train_eval, y_train_eval, param_grid_alpha)

Best parameters: {'alpha': 10.0}
Best accuracy on evaluation set: 0.5086063469241039


In [7]:
param_grid_xgb = {
    'max_depth': [3, 6, 10],
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2]
}

xgb_best, best_params = optimize_and_train_xgb(X_train, y_train, X_eval, y_eval, param_grid_xgb)


C:\Users\Oscar Azrak\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


Best hyperparameters: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 50}


In [8]:

param_grid_lgb = {
    'max_depth': [3, 6, 10],
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'num_leaves': [31, 63, 127, 255]
}

# Call the function with your datasets and hyperparameter grid
lgb_best, best_params = optimize_and_train_lgb(X_train, y_train, X_eval, y_eval, param_grid_lgb)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 47869, number of negative: 49501
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.008679 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8670
[LightGBM] [Info] Number of data points in the train set: 97370, number of used features: 34
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.491620 -> initscore=-0.033525
[LightGBM] [Info] Start training from score -0.033525
Best hyperparameters: {'learning_rate': 0.01, 'max_depth': 10, 'n_estimators': 200, 'num_leaves': 31}
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Wa

In [9]:
NN_model, history, X_test_scaled = train_and_evaluate_NN(X_train_eval, y_train_eval, X_eval, y_eval, X_test, y_test)


C:\Users\Oscar Azrak\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
6086/6086 ━━━━━━━━━━━━━━━━━━━━ 7s 705us/step - accuracy: 0.5193 - loss: 0.2496 - val_accuracy: 0.5444 - val_loss: 0.2475
Epoch 2/50
6086/6086 ━━━━━━━━━━━━━━━━━━━━ 4s 676us/step - accuracy: 0.5342 - loss: 0.2483 - val_accuracy: 0.5472 - val_loss: 0.2471
Epoch 3/50
6086/6086 ━━━━━━━━━━━━━━━━━━━━ 4s 666us/step - accuracy: 0.5428 - loss: 0.2474 - val_accuracy: 0.5519 - val_loss: 0.2462
Epoch 4/50
6086/6086 ━━━━━━━━━━━━━━━━━━━━ 4s 698us/step - accuracy: 0.5434 - loss: 0.2470 - val_accuracy: 0.5520 - val_loss: 0.2461
Epoch 5/50
6086/6086 ━━━━━━━━━━━━━━━━━━━━ 4s 666us/step - accuracy: 0.5495 - loss: 0.2461 - val_accuracy: 0.5590 - val_loss: 0.2451
Epoch 6/50
6086/6086 ━━━━━━━━━━━━━━━━━━━━ 4s 686us/step - accuracy: 0.5542 - loss: 0.2454 - val_accuracy: 0.5621 - val_loss: 0.2442
Epoch 7/50
6086/6086 ━━━━━━━━━━━━━━━━━━━━ 4s 686us/step - accuracy: 0.5573 - loss: 0.2448 - val_accuracy: 0.5649 - val_loss: 0.2433
Epoch 8/50
6086/6086 ━━━━━━━━━━━━━━━━━━━━ 4s 668us/step - accuracy: 0.5630 -

In [10]:

# Store predictions in a dictionary
predictions = {
    "Ridge Classifier": ridge_best.predict(X_test),
    "XGBoost Classifier": xgb_best.predict(X_test),
    "LightGBM Classifier": lgb_best.predict(X_test),
}   

# Iterate through the dictionary and evaluate each model
for model_name, y_pred in predictions.items():
    print(model_name + ":")
    evaluate_model_performance(y_test, y_pred)
    print("\n")


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Ridge Classifier:
Accuracy: 0.5281781630148452
Confusion Matrix:
[[70783 28420]
 [63464 32076]]
Precision: 0.5302168738428987
Recall: 0.33573372409462005
F1 Score: 0.41113589171729603
MSE: 0.4718218369851548
RMSE: 0.6868928861075465


XGBoost Classifier:
Accuracy: 0.5266325362143954
Confusion Matrix:
[[58120 41083]
 [51102 44438]]
Precision: 0.5196150653055974
Recall: 0.46512455516014234
F1 Score: 0.490862195613633
MSE: 0.4733674637856046
RMSE: 0.6880170519584559


LightGBM Classifier:
Accuracy: 0.5251279891960173
Confusion Matrix:
[[59317 39886]
 [52592 42948]]
Precision: 0.5184827486297897
Recall: 0.4495289930918987
F1 Score: 0.48155000168185946
MSE: 0.47487201080398267
RMSE: 0.6891095782268468




In [14]:
# lgb score predict
y_pred_prob_lgb = lgb_best.predict_proba(X_test)[:, 1]  # Get probability of class 1
print("LightGBM Classifier Prediction Scores:", y_pred_prob_lgb)

y_pred_prob_xgb = xgb_best.predict_proba(X_test)[:, 1]  # Get probability of class 1
print("XGBoost Classifier Prediction Scores:", y_pred_prob_xgb)

y_scores = ridge_best.decision_function(X_test)
y_pred_prob_ridge = sigmoid(y_scores)  # Convert scores to probabilities using sigmoid
print("Ridge Classifier Prediction Scores:", y_pred_prob_ridge)

y_pred_prob_NN = NN_model.predict(X_test_scaled)  # Get probability of class 1
print("NN Classifier Prediction Scores:", y_pred_prob_NN)




[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
LightGBM Classifier Prediction Scores: [0.52707745 0.53107638 0.42127966 ... 0.54653081 0.48889907 0.50623161]
XGBoost Classifier Prediction Scores: [0.5330586  0.54325885 0.42294225 ... 0.52330583 0.4944392  0.46730632]
Ridge Classifier Prediction Scores: [0.50229653 0.50292907 0.48869302 ... 0.50448351 0.49014079 0.47994537]
6086/6086 ━━━━━━━━━━━━━━━━━━━━ 2s 406us/step
NN Classifier Prediction Scores: [[0.5182142 ]
 [0.74977434]
 [0.2878918 ]
 ...
 [0.36692137]
 [0.73923874]
 [0.7057015 ]]


In [25]:
X_test['lgb'] = y_pred_prob_lgb
X_test['xgb'] = y_pred_prob_xgb
X_test['ridge'] = y_pred_prob_ridge
X_test['NN'] = y_pred_prob_NN

In [26]:
print(X_test.columns)

Index(['VaR_10', 'VaR_100', 'VaR_180', 'VaR_30', 'VaR_60', 'avgreturn_10',
       'avgreturn_100', 'avgreturn_180', 'avgreturn_30', 'avgreturn_60',
       'momentum_10', 'momentum_100', 'momentum_180', 'momentum_30',
       'momentum_60', 'skew_10', 'skew_100', 'skew_180', 'skew_30', 'skew_60',
       'volatility_10', 'volatility_100', 'volatility_180', 'volatility_30',
       'volatility_60', 'MACRO_8', 'MACRO_0', 'MACRO_1', 'MACRO_2', 'MACRO_3',
       'MACRO_4', 'MACRO_5', 'MACRO_6', 'MACRO_7', 'lgb', 'xgb', 'ridge',
       'NN'],
      dtype='object')


In [27]:
# top 10 best and worst assets for each class and only keep asset column

top_10_best_assets_lgb = X_test.sort_values(by='lgb', ascending=False).head(10)
top_10_worst_assets_lgb = X_test.sort_values(by='lgb', ascending=True).head(10)

top_10_best_assets_xgb = X_test.sort_values(by='xgb', ascending=False).head(10)
top_10_worst_assets_xgb = X_test.sort_values(by='xgb', ascending=True).head(10)

top_10_best_assets_ridge = X_test.sort_values(by='ridge', ascending=False).head(10)
top_10_worst_assets_ridge = X_test.sort_values(by='ridge', ascending=True).head(10)

top_10_best_assets_NN = X_test.sort_values(by='NN', ascending=False).head(10)
top_10_worst_assets_NN = X_test.sort_values(by='NN', ascending=True).head(10)

In [34]:
# print the assets

print("Top 10 best assets for LightGBM Classifier:")
print(top_10_best_assets_lgb.index)
print("\n")
print("Top 10 worst assets for LightGBM Classifier:")
print(top_10_worst_assets_lgb.index)
print("\n")

print("Top 10 best assets for XGBoost Classifier:")
print(top_10_best_assets_xgb.index)
print("\n")
print("Top 10 worst assets for XGBoost Classifier:")
print(top_10_worst_assets_xgb.index)
print("\n")

print("Top 10 best assets for Ridge Classifier:")
print(top_10_best_assets_ridge.index)
print("\n")
print("Top 10 worst assets for Ridge Classifier:")
print(top_10_worst_assets_ridge.index)
print("\n")

print("Top 10 best assets for NN Classifier:")
print(top_10_best_assets_NN.index)
print("\n")
print("Top 10 worst assets for NN Classifier:")
print(top_10_worst_assets_NN.index)
print("\n")



Top 10 best assets for LightGBM Classifier:
Index([425846, 425672, 425498, 425556, 425614, 425730, 425788, 418107, 418165,
       418223],
      dtype='int64')


Top 10 worst assets for LightGBM Classifier:
Index([425785, 425959, 425843, 426133, 425901, 426017, 461778, 461604, 373092,
       425321],
      dtype='int64')


Top 10 best assets for XGBoost Classifier:
Index([332212, 332270, 332328, 434318, 434260, 434376, 434202, 332386, 361143,
       332444],
      dtype='int64')


Top 10 worst assets for XGBoost Classifier:
Index([416824, 416534, 416708, 416592, 416766, 417172, 416882, 416940, 416650,
       461662],
      dtype='int64')


Top 10 best assets for Ridge Classifier:
Index([488762, 460001, 459943, 488704, 490386, 488646, 490444, 458957, 488530,
       460175],
      dtype='int64')


Top 10 worst assets for Ridge Classifier:
Index([458203, 458261, 458319, 458087, 458145, 487950, 459569, 459566, 459565,
       459578],
      dtype='int64')


Top 10 best assets for NN Classif